<a href="https://colab.research.google.com/github/kashifalikhan36/Mental_Health_Explore/blob/main/Mental_Health_explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sample=pd.read_csv("/kaggle/input/jhfjuy7869/sample_submission.csv")
train=pd.read_csv("/kaggle/input/jhfjuy7869/train (1).csv")
test=pd.read_csv("/kaggle/input/jhfjuy7869/test (1).csv")
acc_mod={}

In [ ]:
def cleaning_stuff(train):
    if "id" in train.columns:
        train.drop("id", axis=1, inplace=True)
    list_obj = train.select_dtypes("object").columns.to_list()
    asd = []
    for col in list_obj:
        unique_count = len(train[col].unique())
        if unique_count < 3:
            asd.append(col)
    for i in asd:
        k = pd.get_dummies(train[i], prefix=i).astype(int)
        train = pd.concat([train, k], axis=1)
    train.drop(list_obj, axis=1, inplace=True)
    for i in train.isna().columns.to_list():
        if train[i].isna().sum()>0:
            train[i].fillna(int(train[i].mean()), inplace=True)
    return train

In [ ]:
clean_train=cleaning_stuff(train)
clean_test=cleaning_stuff(test)

/tmp/ipykernel_24/2372484662.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[i].fillna(int(train[i].mean()), inplace=True)
/tmp/ipykernel_24/2372484662.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [ ]:
clean_train.corr()

,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress,Depression,Gender_Female,Gender_Male,Working Professional or Student_Student,Working Professional or Student_Working Professional,Have you ever had suicidal thoughts ?_No,Have you ever had suicidal thoughts ?_Yes,Family History of Mental Illness_No,Family History of Mental Illness_Yes
Age,1.000000,-0.067091,0.101940,-0.217675,-0.307626,0.194716,-0.118356,-0.083569,-0.564671,0.006890,-0.006890,-0.584984,0.584984,0.141502,-0.141502,-0.009409,0.009409
Academic Pressure,-0.067091,1.000000,-0.027720,0.011061,-0.046419,-0.026918,0.051528,0.073474,0.316944,0.009216,-0.009216,0.091827,-0.091827,-0.124558,0.124558,-0.012090,0.012090
Work Pressure,0.101940,-0.027720,1.000000,-0.112685,-0.159331,0.054717,-0.037620,0.004686,-0.025821,0.002957,-0.002957,-0.301598,0.301598,0.000156,-0.000156,0.002216,-0.002216
CGPA,-0.217675,0.011061,-0.112685,1.000000,0.160259,-0.109408,0.044710,0.022489,0.206025,-0.017594,0.017594,0.373383,-0.373383,-0.054838,0.054838,0.006586,-0.006586
Study Satisfaction,-0.307626,-0.046419,-0.159331,0.160259,1.000000,-0.154718,0.048227,0.003032,0.193906,0.002511,-0.002511,0.527973,-0.527973,-0.042314,0.042314,0.008391,-0.008391
Job Satisfaction,0.194716,-0.026918,0.054717,-0.109408,-0.154718,1.000000,-0.059222,-0.040349,-0.255161,0.008624,-0.008624,-0.292918,0.292918,0.087582,-0.087582,0.006987,-0.006987
Work/Study Hours,-0.118356,0.051528,-0.037620,0.044710,0.048227,-0.059222,1.000000,0.036111,0.191746,-0.006957,0.006957,0.116709,-0.116709,-0.064541,0.064541,-0.011882,0.011882
Financial Stress,-0.083569,0.073474,0.004686,0.022489,0.003032,-0.040349,0.036111,1.000000,0.227227,0.008019,-0.008019,0.053051,-0.053051,-0.087884,0.087884,-0.004047,0.004047
Depression,-0.564671,0.316944,-0.025821,0.206025,0.193906,-0.255161,0.191746,0.227227,1.000000,-0.008144,0.008144,0.520790,-0.520790,-0.349066,0.349066,-0.016502,0.016502
Gender_Female,0.006890,0.009216,0.002957,-0.017594,0.002511,0.008624,-0.006957,0.008019,-0.008144,1.000000,-1.000000,-0.006657,0.006657,-0.010864,0.010864,-0.015499,0.015499


In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(clean_train.drop("Depression",axis=1).values)
y=clean_train[["Depression"]].values.reshape(-1,1)
clean_test=sc.fit_transform(clean_test)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
def check_accuracy(y_pred,y_orignal):
    return accuracy_score(y_pred,y_orignal)

In [ ]:
test=pd.read_csv("/kaggle/input/jhfjuy7869/test (1).csv")
id_test=test["id"].values
pred_test_y=lg.predict(clean_test)
sub_data=pd.DataFrame({"id":id_test,"Depression":pred_test_y})
# submission_logitic=pd.concat([id_test,sub_data]) # here we have some NaN values

In [ ]:
from sklearn.linear_model import LogisticRegression
lg=LogisticRegression(fit_intercept=True)
lg.fit(X_train,y_train)
lg.fit(X_test,y_test)
# y_pred=rfc.predict(X_test)
y_pred_rfc=lg.predict(clean_test)
submission_lg=pd.DataFrame({
    "id":id_test,
    "Depression":y_pred_rfc
})
# y_pred=lg.predict(X_test)
# acc_mod["Logistic"]=check_accuracy(y_pred,y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)
rfc.fit(X_test,y_test)
# y_pred=rfc.predict(X_test)
y_pred_rfc=rfc.predict(clean_test)
submission_rfc=pd.DataFrame({
    "id":id_test,
    "Depression":y_pred_rfc
})
# acc_mod["Randomforestclassifier"]=check_accuracy(y_pred,y_test)

/tmp/ipykernel_24/1925301111.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train,y_train)
/tmp/ipykernel_24/1925301111.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_test,y_test)


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier()
ada.fit(X_train,y_train)
ada.fit(X_test,y_test)
# y_pred=rfc.predict(X_test)
y_pred_rfc=ada.predict(clean_test)
submission_ada=pd.DataFrame({
    "id":id_test,
    "Depression":y_pred_rfc
})
# y_pred=ada.predict(X_test)
# acc_mod["AdaBoot"]=check_accuracy(y_pred,y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
import lightgbm as lgb
lgbc=lgb.LGBMClassifier()
lgbc.fit(X_train,y_train)
lgbc.fit(X_test,y_test)
# y_pred=rfc.predict(X_test)
y_pred_rfc=lgbc.predict(clean_test)
submission_lgbc=pd.DataFrame({
    "id":id_test,
    "Depression":y_pred_rfc
})
# y_pred=lgbc.predict(X_test)
# acc_mod["LGBM"]=check_accuracy(y_pred,y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 20413, number of negative: 92147
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010398 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 356
[LightGBM] [Info] Number of data points in the train set: 112560, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.181352 -> initscore=-1.507213
[LightGBM] [Info] Start training from score -1.507213


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 5154, number of negative: 22986
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002658 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 357
[LightGBM] [Info] Number of data points in the train set: 28140, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.183156 -> initscore=-1.495112
[LightGBM] [Info] Start training from score -1.495112


In [ ]:
import catboost as cb
cbc=cb.CatBoostClassifier()
cbc.fit(X_train,y_train)

cbc.fit(X_test,y_test)
y_pred_rfc=cbc.predict(clean_test)
submission_cbc=pd.DataFrame({
    "id":id_test,
    "Depression":y_pred_rfc
})
# y_pred=cbc.predict(X_test)
# acc_mod["CAT"]=check_accuracy(y_pred,y_test)

Learning rate set to 0.077424
0:	learn: 0.5740773	total: 14.4ms	remaining: 14.4s
1:	learn: 0.4802775	total: 29.4ms	remaining: 14.7s
2:	learn: 0.4116371	total: 44.6ms	remaining: 14.8s
3:	learn: 0.3599294	total: 59.6ms	remaining: 14.8s
4:	learn: 0.3208009	total: 75.3ms	remaining: 15s
5:	learn: 0.2926697	total: 91.1ms	remaining: 15.1s
6:	learn: 0.2700230	total: 107ms	remaining: 15.2s
7:	learn: 0.2523726	total: 122ms	remaining: 15.2s
8:	learn: 0.2386488	total: 138ms	remaining: 15.1s
9:	learn: 0.2275239	total: 153ms	remaining: 15.2s
10:	learn: 0.2194619	total: 168ms	remaining: 15.1s
11:	learn: 0.2124634	total: 183ms	remaining: 15.1s
12:	learn: 0.2079225	total: 197ms	remaining: 15s
13:	learn: 0.2023321	total: 213ms	remaining: 15s
14:	learn: 0.1972946	total: 228ms	remaining: 15s
15:	learn: 0.1934737	total: 243ms	remaining: 15s
16:	learn: 0.1904061	total: 258ms	remaining: 14.9s
17:	learn: 0.1881970	total: 274ms	remaining: 14.9s
18:	learn: 0.1863479	total: 288ms	remaining: 14.9s
19:	learn: 0.18

In [ ]:
from sklearn.svm import SVC
svm=SVC()
svm.fit(X_train,y_train)
svm.fit(X_test,y_test)
y_pred_rfc=svm.predict(clean_test)
submission_svm=pd.DataFrame({
    "id":id_test,
    "Depression":y_pred_rfc
})
# y_pred=svm.predict(X_test)
# acc_mod["SVM"]=check_accuracy(y_pred,y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# acc_mod

In [ ]:
submission_rfc.to_csv("/kaggle/working/submission_rfc.csv",index=False)
submission_ada.to_csv("/kaggle/working/submission_ada.csv",index=False)
submission_svm.to_csv("/kaggle/working/submission_svm.csv",index=False)
submission_lg.to_csv("/kaggle/working/submission_lg.csv",index=False)
submission_cbc.to_csv("/kaggle/working/submission_cbc.csv",index=False)
submission_lgbc.to_csv("/kaggle/working/submission_lgbmc.csv",index=False)